# 다음 코드에서는 SVD모델를 활용한 빈칸을 채워 평균평점을 출력하는 코드를 작성하였음.

In [1]:
# -*- coding: utf-8 -*-

%matplotlib inline

import time
import operator

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [12]:
# Data Source : https://grouplens.org/datasets/movielens
## MovieLens 데이터셋의 기본 정보
rating_file_path = "../data/ml-1m/ratings.dat"
movie_file_path = "../data/ml-1m/movies.dat"
user_file_path = "../data/ml-1m/users.dat"

rating_data = pd.io.parsers.read_csv(rating_file_path, 
                                     names=['user_id', 'movie_id', 'rating', 'time'], delimiter='::')
movie_data = pd.io.parsers.read_csv(movie_file_path, 
                                    names=['movie_id', 'title', 'genre'], delimiter='::')
user_data = pd.io.parsers.read_csv(user_file_path, 
                                   names=['user_id', 'gender', 'age', 'occupation', 'zipcode'], delimiter='::') 

In [11]:
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
#Surprise 라이브러리를 사용하여 SVD 모델을 초기화하고, 
#데이터를 학습 데이터와 테스트 데이터로 분리한 뒤 모델을 훈련하고 예측을 수행

np.set_printoptions(suppress=True)

reader = Reader(rating_scale=(1, 10))

In [13]:
# 데이터 프레임을 생성합니다.
ratings_dict = {'item_id': [1,1,1,2,2,3,3,3,4],
                'user_id': ['a','b','d','a','c','a','b','d','c'],
                'rating': [8,6,7,9,8,8,7,8,8]}
df = pd.DataFrame(ratings_dict)

In [14]:
# SVD 사용을 위한 데이터셋을 생성합니다.
data = Dataset.load_from_df(df=df[["user_id", "item_id", "rating"]], reader=reader)
train_data = data.build_full_trainset()

In [15]:
# Surprise 라이브러리를 위한 데이터셋 생성
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(rating_data[['user_id', 'movie_id', 'rating']], reader)

# SVD 모델 초기화 및 학습
model = SVD()
trainset = data.build_full_trainset()
model.fit(trainset)

# 4번 사용자가 안 본 액션 영화들을 추출합니다.
unseen_action_movies = movie_data.loc[~movie_data['movie_id'].isin(rating_data.loc[rating_data['user_id'] == 4, 'movie_id']), 'movie_id']

# 4번 사용자가 안 본 액션 영화들에 대한 평균 평점을 예측합니다.
predictions = []
for movie_id in unseen_action_movies:
    prediction = model.predict(4, movie_id)
    predictions.append((movie_id, prediction.est))

# 예측 결과를 데이터프레임으로 변환하여 출력합니다.
prediction_df = pd.DataFrame(predictions, columns=['movie_id', 'predicted_rating'])
print(prediction_df)

      movie_id  predicted_rating
0            1          4.515219
1            2          3.280705
2            3          3.163209
3            4          2.931817
4            5          3.267238
...        ...               ...
3857      3948          3.843249
3858      3949          4.115211
3859      3950          3.974388
3860      3951          4.194489
3861      3952          4.131510

[3862 rows x 2 columns]
